In [2]:
#Import libraries

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import warnings
import random
warnings.filterwarnings('ignore')
import sklearn
from sklearn.ensemble import RandomForestClassifier

import sys
sys.path
sys.path.append('/Users/matthewmcteer/opt/anaconda3/lib/python3.7/site-packages')

import lightgbm
import shap
import xgboost as xgb

from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, GridSearchCV
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

import time
import missingno as msno

import lightgbm as lgbm



# Response 11 - MASL vs MASH

In [10]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

In [ ]:
features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
           'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'waist_to_hip_ratio',
            'idf_metabolic_syndrome',
            'eGFR',
            'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_IGA',
            'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            'FIB4',
            'NFS',
            'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            'BARD',
            'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_11'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_11 = NaN
df = df[features]
df = df[df['response_11'].notna()]

df['response_11'].value_counts()

In [12]:
#specialist cohort index numbers

response11_idx = [156, 218, 224, 230, 231, 235, 251, 256, 258, 494, 500, 508, 513, 518, 524, 529, 533, 541, 545, 551, 555, 566, 567, 705, 714, 716, 721, 722, 730, 733, 738, 745, 759, 762, 803, 814, 849, 867, 871, 905, 906, 910, 912, 948, 962, 965, 996, 1032, 1034, 1038, 1039, 1042, 1044, 1047, 1051, 1054, 1055, 1056, 1059, 1087, 1106, 1143, 1144, 1162, 1185, 1219, 1273, 1305, 1308, 1309, 1322, 1327, 1371, 1372, 1374, 1379, 1380, 1385, 1390, 1391, 1395, 1396, 1399, 2088, 2092, 2095, 2098, 2101, 2105, 4126, 4202, 4205, 4445, 4521, 4522, 4523, 4526, 4529, 4531, 4532, 4534, 4535, 4537, 4538, 4539, 4540, 4542, 4543, 4544, 4546, 4547, 4550, 4551, 4552, 4554, 4555, 4556, 4558, 4559, 4560, 4561, 4565, 4568, 4569, 4571, 4574, 4575, 4577, 4579, 4581, 4589, 4593, 4596, 4601, 4608, 4615, 4617, 4618, 4619, 4621, 4623, 4626, 4627, 4629, 4638, 4639, 4641, 4643, 4646, 4647, 4653, 4656, 4657, 4659, 4660, 4661, 4662, 4664, 4665, 4683, 4684, 4688, 4692, 4693, 4695, 4696, 5169, 5174, 5176, 5178, 5184, 5195, 5196, 5211, 5214, 5215, 5216, 5219, 5224, 5227, 5229, 5230, 5235, 5243, 5256, 5259, 5273, 5275, 5281, 5287, 5294, 5297, 5308, 5330, 5341, 5345, 5347, 5354, 5363, 5364, 5374, 5375, 5377, 5391, 5393, 5414, 5419, 5449, 5484, 5485, 5502, 5504, 5507, 5509, 5510, 5511, 5513, 5514, 5516, 5521, 5522, 5524, 5530, 5533, 5535, 5537, 5539, 5545, 5550, 5554, 5555, 5556, 5562, 5563, 5566, 5571, 5574, 5580, 5585, 5586, 5587, 5589, 5594, 5595, 5600, 5602, 5608, 5610, 5618, 5622, 5627, 5629, 5630, 5632, 5634, 5635, 5639, 5640, 5643, 5644, 5645, 5646, 5647, 5648, 5785, 5790, 5814, 5827, 5830, 5836, 5840, 5845, 5852, 5866, 5870, 5876, 5901, 5909, 5914, 5920, 5930, 5934, 5939, 5944, 5964, 5970, 5972, 5981, 5986, 5997, 5998, 6006, 6015, 6016, 6018, 6022, 6031, 6033, 6035, 6037, 6045, 6047, 6053, 6054, 6056, 6060, 6067, 6072, 6079, 6087, 6093, 6099, 6108, 6119, 6126, 6134, 6158, 6162, 6168, 6179, 6200, 6204, 6227, 6236, 6237, 6250, 6262, 6297, 6299, 6307, 6317, 6320, 6327, 6328, 6329, 6331, 6348, 6353, 6354, 6365, 6367, 6368, 6373, 6374, 6390, 6401, 6403, 6409, 6414, 6423, 6424, 6430, 6436, 6437, 6440, 6449, 6463, 6465, 6469, 6503, 6507, 6513, 6521, 6532, 6537, 6545, 6550, 6555, 6562, 6572, 6587, 6593, 6629, 6635, 6664, 6669, 6674, 6681, 6693, 6703, 6713, 6718, 6723, 6726, 6736, 6771, 6777, 6789, 6792, 6803, 6830, 6835, 6858, 6862, 6871, 6885, 6889, 6893, 6896, 6909, 6914, 6921, 6929, 6942, 6953, 6959, 6984, 6987, 7010, 7030, 7033, 7064, 7067, 7072, 7078, 7080, 7088, 7096, 7099, 7107, 7109, 7112, 7126, 7160, 7193, 7197, 7204, 7208, 7210, 7211, 7225, 7226, 7229, 7237, 7250, 7251, 8285, 8287, 8290, 8293, 8301, 8308, 8314, 8316, 8323, 8329, 8339, 8349, 8350, 8352, 8353, 8354, 8355, 8360, 8362, 8363, 8364, 8366, 8367, 8368, 8369, 8370, 8371, 8372, 8373, 8376, 8379, 8380, 8381, 8382, 8383, 8386, 8391, 8392, 8394, 8398, 8404, 8411, 8412, 8413, 8418, 8420, 8421, 8422, 8423, 8424, 8425, 8430, 8431, 8432, 8435, 8436, 8438, 8442, 8452, 8457, 8463, 8464, 8475, 8478, 8496, 8497, 8501, 8506, 8510, 8513, 8518, 8520, 8525, 8539, 8541, 8544, 8547, 8550, 8553, 8556, 8560, 8561, 8563, 8566, 8568, 8572, 8575, 8577, 8580, 8583, 8584, 8585, 8588, 8590, 8592, 8595, 8597, 8599, 8602, 8606, 8607, 8609, 8611, 8613, 8614, 8616, 8618, 8619, 8622, 8625, 8627, 8630, 8632, 8637, 8639, 8641, 8643, 8644, 8658, 8678, 8680, 8685, 8693, 8696, 8700, 8701, 8706, 8708, 8709, 8711, 8713, 8717, 8722, 8724, 8725, 8728, 8729, 8730, 8733, 8736, 8737, 8741, 8742, 8745, 8747, 8750, 9084, 9085, 9086, 9087, 9088, 9090, 9091, 9102, 9103, 9104, 9107, 9110, 9112, 9113, 9114, 9115, 9116, 9117, 9118, 9119, 9121, 9123, 9125, 9126, 9128, 9129, 9130, 9131, 9134, 9135, 9139, 9140, 9142, 9148, 9150, 9151, 9152, 9153, 9156]

In [ ]:
df = df.loc[response11_idx]
df = df[df['response_11'].notna()]
df['response_11'].isna().sum()

In [ ]:
df.shape

In [ ]:
#Bar chart displaying level of completeness for each feature, n = 8550
null_values_bar = msno.bar(df,labels=True)
null_values_bar

In [16]:
df.to_csv(#save response 11 specialist data)



# XGBoost

In [ ]:
df = pd.read_csv(#read response 11 specialist data)

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
           'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'waist_to_hip_ratio',
            'idf_metabolic_syndrome',
            'eGFR',
            'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_IGA',
            'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            'FIB4',
            'NFS',
            'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            'BARD',
            'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_11'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_11 = NaN
df = df[features]
df = df[df['response_11'].notna()]

df['response_11'].value_counts()

In [ ]:
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'MASL vs. MASH']
df


In [6]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Create train and test sets  
X = df.iloc[:,:35]
y = df['MASL vs. MASH']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

In [25]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB - Specialist Variables - MASL vs. MASH")
plt.show()



## XGBoost + MICE 

In [ ]:
df = pd.read_csv(#read imputed response 11 specialist data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'MASL vs. MASH']
df.shape

In [13]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Create train and test sets  
X = df.iloc[:,:35]
y = df['MASL vs. MASH']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

In [36]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB + MICE - Specialist Variables - MASL vs. MASH")
plt.show()



## XGBoost + MICE + SMOTE

In [18]:
import imblearn
from imblearn.over_sampling import SMOTE

In [ ]:
df = pd.read_csv(#read imputed response 11 specialist data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'MASL vs. MASH']
df.shape

In [20]:
#Create train and test sets
X = df.iloc[:,:35]
y = df['MASL vs. MASH']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

In [21]:
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

In [ ]:
unique, counts = np.unique(y_train_res, return_counts=True)

print(np.asarray((unique, counts)).T)

In [ ]:
print(y_test.value_counts())

In [24]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train_res, y_train_res)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train_res,y_train_res)

In [53]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train_res, 
    y = y_train_res, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train_res
)

shap_values = explainer.shap_values(X_train_res)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train_res, plot_type="bar",show=False)
plt.title("XGB + MICE + SMOTE - Specialist Variables - MASL vs. MASH")
plt.show()



# Response 12 - At-Risk MASH

In [57]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

In [ ]:
features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
           'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'waist_to_hip_ratio',
            'idf_metabolic_syndrome',
            'eGFR',
            'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_IGA',
            'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            'FIB4',
            'NFS',
            'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            'BARD',
            'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_7'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_12 = NaN
df = df[features]
df = df[df['response_12'].notna()]

df['response_12'].value_counts()

In [59]:
#Specialist cohort index numbers
response12_idx = [156, 218, 224, 230, 231, 235, 251, 256, 258, 494, 500, 508, 513, 518, 524, 529, 533, 541, 545, 551, 555, 566, 567, 705, 714, 716, 721, 722, 730, 733, 738, 745, 759, 762, 803, 814, 849, 867, 871, 905, 906, 910, 912, 948, 962, 965, 996, 1032, 1034, 1038, 1039, 1042, 1044, 1047, 1051, 1054, 1055, 1056, 1059, 1087, 1106, 1143, 1144, 1162, 1185, 1219, 1273, 1305, 1308, 1309, 1322, 1327, 1371, 1372, 1374, 1379, 1380, 1385, 1390, 1391, 1395, 1396, 1399, 2088, 2092, 2095, 2098, 2101, 2105, 4126, 4202, 4205, 4445, 4521, 4522, 4523, 4526, 4529, 4531, 4532, 4534, 4535, 4537, 4538, 4539, 4540, 4542, 4543, 4544, 4546, 4547, 4550, 4551, 4552, 4554, 4555, 4556, 4558, 4559, 4560, 4561, 4565, 4568, 4569, 4571, 4574, 4575, 4577, 4579, 4581, 4589, 4593, 4596, 4601, 4608, 4615, 4617, 4618, 4619, 4621, 4623, 4626, 4627, 4629, 4638, 4639, 4641, 4643, 4646, 4647, 4653, 4656, 4657, 4659, 4660, 4661, 4662, 4664, 4665, 4683, 4684, 4688, 4692, 4693, 4695, 4696, 5169, 5174, 5176, 5178, 5184, 5195, 5196, 5211, 5214, 5215, 5216, 5219, 5224, 5227, 5229, 5230, 5235, 5243, 5256, 5259, 5273, 5275, 5281, 5287, 5294, 5297, 5308, 5330, 5341, 5345, 5347, 5354, 5363, 5364, 5374, 5375, 5377, 5391, 5393, 5414, 5419, 5449, 5484, 5485, 5502, 5504, 5507, 5509, 5510, 5511, 5513, 5514, 5516, 5521, 5522, 5524, 5530, 5533, 5535, 5537, 5539, 5541, 5545, 5550, 5554, 5555, 5556, 5562, 5563, 5566, 5571, 5574, 5580, 5585, 5586, 5587, 5589, 5594, 5595, 5600, 5602, 5608, 5610, 5618, 5622, 5627, 5629, 5630, 5632, 5634, 5635, 5639, 5640, 5643, 5644, 5645, 5646, 5647, 5648, 5785, 5790, 5814, 5821, 5827, 5830, 5836, 5840, 5845, 5852, 5866, 5870, 5876, 5901, 5909, 5914, 5920, 5930, 5934, 5939, 5944, 5964, 5970, 5972, 5981, 5986, 5997, 5998, 6006, 6015, 6016, 6018, 6022, 6031, 6033, 6035, 6037, 6045, 6047, 6053, 6054, 6056, 6060, 6067, 6072, 6076, 6079, 6087, 6093, 6099, 6108, 6119, 6126, 6134, 6158, 6162, 6168, 6179, 6200, 6204, 6227, 6236, 6237, 6250, 6262, 6297, 6299, 6307, 6315, 6317, 6320, 6327, 6328, 6329, 6331, 6348, 6353, 6354, 6365, 6367, 6368, 6373, 6374, 6390, 6401, 6403, 6409, 6414, 6423, 6424, 6430, 6436, 6437, 6440, 6449, 6463, 6465, 6469, 6503, 6507, 6513, 6521, 6532, 6537, 6545, 6550, 6555, 6562, 6572, 6587, 6593, 6629, 6635, 6664, 6669, 6674, 6681, 6693, 6703, 6713, 6718, 6723, 6726, 6736, 6771, 6777, 6789, 6792, 6803, 6830, 6835, 6858, 6862, 6871, 6885, 6889, 6893, 6896, 6909, 6914, 6921, 6929, 6942, 6953, 6959, 6984, 6987, 7001, 7010, 7030, 7033, 7064, 7067, 7072, 7078, 7080, 7088, 7096, 7099, 7107, 7109, 7112, 7126, 7160, 7193, 7197, 7204, 7208, 7210, 7211, 7225, 7226, 7229, 7237, 7250, 7251, 8285, 8287, 8290, 8293, 8301, 8308, 8314, 8316, 8323, 8329, 8339, 8349, 8350, 8352, 8353, 8354, 8355, 8360, 8362, 8363, 8364, 8366, 8367, 8368, 8369, 8370, 8371, 8372, 8373, 8376, 8379, 8380, 8381, 8382, 8383, 8386, 8391, 8392, 8394, 8398, 8404, 8411, 8412, 8413, 8418, 8420, 8421, 8422, 8423, 8424, 8425, 8430, 8431, 8432, 8435, 8436, 8438, 8442, 8452, 8457, 8463, 8464, 8475, 8478, 8496, 8497, 8501, 8506, 8510, 8513, 8518, 8520, 8525, 8539, 8541, 8544, 8547, 8550, 8553, 8556, 8560, 8561, 8563, 8566, 8568, 8572, 8575, 8577, 8580, 8583, 8584, 8585, 8588, 8590, 8592, 8595, 8597, 8599, 8602, 8606, 8607, 8609, 8611, 8613, 8614, 8616, 8618, 8619, 8622, 8625, 8627, 8630, 8632, 8637, 8639, 8641, 8643, 8644, 8658, 8678, 8680, 8685, 8693, 8696, 8700, 8701, 8706, 8708, 8709, 8711, 8713, 8715, 8717, 8722, 8724, 8725, 8728, 8729, 8730, 8733, 8736, 8737, 8741, 8742, 8745, 8747, 8750, 9084, 9085, 9086, 9087, 9088, 9090, 9091, 9102, 9103, 9104, 9107, 9110, 9112, 9113, 9114, 9115, 9116, 9117, 9118, 9119, 9121, 9123, 9125, 9126, 9128, 9129, 9130, 9131, 9134, 9135, 9139, 9140, 9142, 9148, 9150, 9151, 9152, 9153, 9156]

In [ ]:
df = df.loc[response12_idx]
df = df[df['response_7'].notna()]
df['response_7'].isna().sum()

In [ ]:
df.shape

In [ ]:
#Bar chart displaying level of completeness for each feature, n = 8550
null_values_bar = msno.bar(df,labels=True)
null_values_bar

In [64]:
df.to_csv(#save response 12 specialist data)



## XGBoost

In [ ]:
df = pd.read_csv(#read specialist response 12 data)

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
           'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'waist_to_hip_ratio',
            'idf_metabolic_syndrome',
            'eGFR',
            'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_IGA',
            'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            'FIB4',
            'NFS',
            'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            'BARD',
            'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_12'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_12 = NaN
df = df[features]
df = df[df['response_12'].notna()]

df['response_12'].value_counts()

In [31]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Create train and test sets  
X = df.iloc[:,:35]
y = df['At-Risk MASH']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

In [150]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB - Specialist Variables - At-Risk MASH")
plt.show()



## XGBoost + MICE

In [ ]:
df = pd.read_csv(#read specialist response 12 imputed data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'At-Risk MASH']
df.shape

In [ ]:
#Bar chart displaying level of completeness for each feature, n = 8550
null_values_bar = msno.bar(df,labels=True)
null_values_bar

In [38]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Create train and test sets  
X = df.iloc[:,:35]
y = df['At-Risk MASH']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

In [161]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB + MICE - Specialist Variables - At-Risk MASH")
plt.show()



## XGBoost + MICE + SMOTE

In [43]:
import imblearn
from imblearn.over_sampling import SMOTE

In [ ]:
df = pd.read_csv(#read imputed response 12 specialist data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'At-Risk MASH']
df.shape

In [45]:
#Create train and test sets
X = df.iloc[:,:35]
y = df['At-Risk MASH']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

In [46]:
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

In [ ]:
unique, counts = np.unique(y_train_res, return_counts=True)

print(np.asarray((unique, counts)).T)

In [ ]:
print(y_test.value_counts())

In [49]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train_res, y_train_res)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train_res,y_train_res)

In [175]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train_res, 
    y = y_train_res, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
from matplotlib.pyplot import figure
import sklearn.metrics as metrics
figure(figsize=(8, 6), dpi=80)
from sklearn.metrics import roc_curve
from scipy import interp
from sklearn.metrics import auc
cv = StratifiedKFold(n_splits=10,shuffle=False)

#X = X_train_res
#y = y_train_res
tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)
i = 1
for Train,Test in cv.split(X_train_res,y_train_res):
    prediction = model.fit(X_train_res.iloc[Train],y_train_res[Train]).predict_proba(X_train_res.iloc[Test])
    fpr, tpr, t = roc_curve(y_train_res[Test], prediction[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=2, alpha=0.1, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
    i= i+1
plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean K-Fold C.V ROC (AUC = %0.2f )' % (mean_auc),lw=2, alpha=1)
plt.plot()

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC for k-Fold Cross-Valdiation (k=10)')
plt.legend(loc="lower right")

#plt.text(0.32,0.7,'More accurate area',fontsize = 12)
#plt.text(0.63,0.4,'Less accurate area',fontsize = 12)
#plt.show() 

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = sklearn.metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('ROC for K-Fold C.V and Test Set')
plt.plot(fpr, tpr, 'm', label =r'ROC Test Set (AUC = %0.2f)' % roc_auc, lw = 2, alpha = 1)
plt.legend(loc = 'lower right')
#plt.plot([0, 1], [0, 1],'k--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train_res
)

shap_values = explainer.shap_values(X_train_res)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train_res, plot_type="bar",show=False)
plt.title("XGB + MICE + SMOTE - Specialist Variables - At-Risk MASH")
plt.show()



# Response 13 - High Activity

In [181]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

In [ ]:
features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
           'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'waist_to_hip_ratio',
            'idf_metabolic_syndrome',
            'eGFR',
            'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_IGA',
            'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            'FIB4',
            'NFS',
            'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            'BARD',
            'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_13'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_13 = NaN
df = df[features]
df = df[df['response_13'].notna()]

df['response_13'].value_counts()

In [183]:
#specialist cohort index numbers

response13_idx = [156, 218, 224, 230, 231, 235, 251, 256, 258, 494, 500, 508, 513, 518, 524, 529, 533, 541, 545, 551, 555, 566, 567, 705, 714, 716, 721, 722, 730, 733, 738, 745, 759, 762, 803, 814, 849, 867, 871, 905, 906, 910, 912, 948, 962, 965, 996, 1032, 1034, 1038, 1039, 1042, 1044, 1047, 1051, 1054, 1055, 1056, 1059, 1087, 1106, 1143, 1144, 1162, 1185, 1219, 1273, 1305, 1308, 1309, 1322, 1327, 1371, 1372, 1374, 1379, 1380, 1385, 1390, 1391, 1395, 1396, 1399, 2088, 2092, 2095, 2098, 2101, 2105, 4126, 4202, 4205, 4445, 4521, 4522, 4523, 4526, 4529, 4531, 4532, 4534, 4535, 4537, 4538, 4539, 4540, 4542, 4543, 4544, 4546, 4547, 4550, 4551, 4552, 4554, 4555, 4556, 4558, 4559, 4560, 4561, 4565, 4568, 4569, 4571, 4574, 4575, 4577, 4579, 4581, 4589, 4593, 4596, 4601, 4608, 4615, 4617, 4618, 4619, 4621, 4623, 4626, 4627, 4629, 4638, 4639, 4641, 4643, 4646, 4647, 4653, 4656, 4657, 4659, 4660, 4661, 4662, 4664, 4665, 4683, 4684, 4688, 4692, 4693, 4695, 4696, 5169, 5174, 5176, 5178, 5184, 5195, 5196, 5211, 5214, 5215, 5216, 5219, 5224, 5227, 5229, 5230, 5235, 5243, 5256, 5259, 5273, 5275, 5281, 5287, 5294, 5297, 5308, 5330, 5341, 5345, 5347, 5354, 5363, 5364, 5374, 5375, 5377, 5391, 5393, 5414, 5419, 5449, 5484, 5485, 5502, 5504, 5507, 5509, 5510, 5511, 5513, 5514, 5516, 5521, 5522, 5524, 5530, 5533, 5535, 5537, 5539, 5545, 5550, 5554, 5555, 5556, 5562, 5563, 5566, 5571, 5574, 5580, 5585, 5586, 5587, 5589, 5594, 5595, 5600, 5602, 5608, 5610, 5618, 5622, 5627, 5629, 5630, 5632, 5634, 5635, 5639, 5640, 5643, 5644, 5645, 5646, 5647, 5648, 5785, 5790, 5814, 5827, 5830, 5836, 5840, 5845, 5852, 5866, 5870, 5876, 5901, 5909, 5914, 5920, 5930, 5934, 5939, 5944, 5964, 5970, 5972, 5981, 5986, 5997, 5998, 6006, 6015, 6016, 6018, 6022, 6031, 6033, 6035, 6037, 6045, 6047, 6053, 6054, 6056, 6060, 6067, 6072, 6079, 6087, 6093, 6099, 6108, 6119, 6126, 6134, 6158, 6162, 6168, 6179, 6200, 6204, 6227, 6236, 6237, 6250, 6262, 6297, 6299, 6307, 6317, 6320, 6327, 6328, 6329, 6331, 6348, 6353, 6354, 6365, 6367, 6368, 6373, 6374, 6390, 6401, 6403, 6409, 6414, 6423, 6424, 6430, 6436, 6437, 6440, 6449, 6463, 6465, 6469, 6503, 6507, 6513, 6521, 6532, 6537, 6545, 6550, 6555, 6562, 6572, 6587, 6593, 6629, 6635, 6664, 6669, 6674, 6681, 6693, 6703, 6713, 6718, 6723, 6726, 6736, 6771, 6777, 6789, 6792, 6803, 6830, 6835, 6858, 6862, 6871, 6885, 6889, 6893, 6896, 6909, 6914, 6921, 6929, 6942, 6953, 6959, 6984, 6987, 7010, 7030, 7033, 7064, 7067, 7072, 7078, 7080, 7088, 7096, 7099, 7107, 7109, 7112, 7126, 7160, 7193, 7197, 7204, 7208, 7210, 7211, 7225, 7226, 7229, 7237, 7250, 7251, 8285, 8287, 8290, 8293, 8301, 8308, 8314, 8316, 8323, 8329, 8339, 8349, 8350, 8352, 8353, 8354, 8355, 8360, 8362, 8363, 8364, 8366, 8367, 8368, 8369, 8370, 8371, 8372, 8373, 8376, 8379, 8380, 8381, 8382, 8383, 8386, 8391, 8392, 8394, 8398, 8404, 8411, 8412, 8413, 8418, 8420, 8421, 8422, 8423, 8424, 8425, 8430, 8431, 8432, 8435, 8436, 8438, 8442, 8452, 8457, 8463, 8464, 8475, 8478, 8496, 8497, 8501, 8506, 8510, 8513, 8518, 8520, 8525, 8539, 8541, 8544, 8547, 8550, 8553, 8556, 8560, 8561, 8563, 8566, 8568, 8572, 8575, 8577, 8580, 8583, 8584, 8585, 8588, 8590, 8592, 8595, 8597, 8599, 8602, 8606, 8607, 8609, 8611, 8613, 8614, 8616, 8618, 8619, 8622, 8625, 8627, 8630, 8632, 8637, 8639, 8641, 8643, 8644, 8658, 8678, 8680, 8685, 8693, 8696, 8700, 8701, 8706, 8708, 8709, 8711, 8713, 8717, 8722, 8724, 8725, 8728, 8729, 8730, 8733, 8736, 8737, 8741, 8742, 8745, 8747, 8750, 9084, 9085, 9086, 9087, 9088, 9090, 9091, 9102, 9103, 9104, 9105, 9107, 9110, 9112, 9113, 9114, 9115, 9116, 9117, 9118, 9119, 9121, 9123, 9125, 9126, 9128, 9129, 9130, 9131, 9134, 9135, 9139, 9140, 9142, 9148, 9150, 9151, 9152, 9153, 9154, 9155, 9156]

In [ ]:
df = df.loc[response13_idx]
df = df[df['response_13'].notna()]
df['response_13'].isna().sum()

In [ ]:
df.shape

In [ ]:
#Bar chart displaying level of completeness for each feature, n = 8550
null_values_bar = msno.bar(df,labels=True)
null_values_bar

In [187]:
df.to_csv('#save specialist response 13 data)



## XGBoost

In [ ]:
df = pd.read_csv(#read specialist response 13 data)

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
           'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'waist_to_hip_ratio',
            'idf_metabolic_syndrome',
            'eGFR',
            'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_IGA',
            'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            'FIB4',
            'NFS',
            'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            'BARD',
            'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_13'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_13 = NaN
df = df[features]
df = df[df['response_13'].notna()]

df['response_13'].value_counts()

In [ ]:
df.shape

In [ ]:
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'High Activity']
df


In [56]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Create train and test sets  
X = df.iloc[:,:35]
y = df['High Activity']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

In [196]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB - Specialist Variables - High Activity")
plt.show()



## XGBoost + MICE 

In [ ]:
df = pd.read_csv(#read imputed specialist response 13 data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'High Activity']
df.shape

In [63]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Create train and test sets  
X = df.iloc[:,:35]
y = df['High Activity']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

In [207]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB + MICE - Specialist Variables - High Activity")
plt.show()



## XGBoost + MICE + SMOTE

In [68]:
import imblearn
from imblearn.over_sampling import SMOTE

In [ ]:
df = pd.read_csv(#read imputed specialist response 13 data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'High Activity']
df.shape

In [70]:
#Create train and test sets
X = df.iloc[:,:35]
y = df['High Activity']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

In [71]:
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

In [ ]:
unique, counts = np.unique(y_train_res, return_counts=True)

print(np.asarray((unique, counts)).T)

In [ ]:
print(y_test.value_counts())

In [74]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train_res, y_train_res)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train_res,y_train_res)

In [221]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train_res, 
    y = y_train_res, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train_res
)

shap_values = explainer.shap_values(X_train_res)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train_res, plot_type="bar",show=False)
plt.title("XGB + MICE + SMOTE - Specialist Variables - High Activity")
plt.show()



# Response 14 - Clinically Significant Fibrosis 

In [225]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

In [ ]:
features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
           'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'waist_to_hip_ratio',
            'idf_metabolic_syndrome',
            'eGFR',
            'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_IGA',
            'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            'FIB4',
            'NFS',
            'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            'BARD',
            'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_14'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_14 = NaN
df = df[features]
df = df[df['response_14'].notna()]

df['response_14'].value_counts()

In [227]:
#response 14 specialist indexes
response14_idx = [156, 218, 224, 230, 231, 235, 251, 256, 258, 494, 500, 508, 513, 518, 524, 529, 533, 541, 545, 551, 555, 566, 567, 705, 714, 716, 721, 722, 730, 733, 738, 745, 759, 762, 803, 814, 849, 867, 871, 905, 906, 910, 912, 948, 962, 965, 996, 1032, 1034, 1038, 1039, 1042, 1044, 1047, 1051, 1054, 1055, 1056, 1059, 1087, 1106, 1143, 1144, 1162, 1185, 1219, 1273, 1305, 1308, 1309, 1322, 1327, 1371, 1372, 1374, 1379, 1380, 1385, 1390, 1391, 1395, 1396, 1399, 2088, 2092, 2095, 2098, 2101, 2105, 4126, 4202, 4205, 4441, 4445, 4521, 4522, 4523, 4526, 4529, 4531, 4532, 4534, 4535, 4537, 4538, 4539, 4540, 4542, 4543, 4544, 4546, 4547, 4550, 4551, 4552, 4554, 4555, 4556, 4558, 4559, 4560, 4561, 4565, 4568, 4569, 4571, 4574, 4575, 4577, 4579, 4581, 4589, 4593, 4596, 4601, 4608, 4615, 4617, 4618, 4619, 4621, 4623, 4626, 4627, 4629, 4638, 4639, 4641, 4643, 4646, 4647, 4653, 4656, 4657, 4659, 4660, 4661, 4662, 4664, 4665, 4683, 4684, 4688, 4692, 4693, 4695, 4696, 5169, 5174, 5176, 5178, 5184, 5195, 5196, 5211, 5214, 5215, 5216, 5219, 5224, 5227, 5229, 5230, 5235, 5243, 5256, 5259, 5273, 5275, 5281, 5287, 5294, 5297, 5308, 5330, 5341, 5345, 5347, 5354, 5363, 5364, 5374, 5375, 5377, 5391, 5393, 5414, 5419, 5449, 5484, 5485, 5502, 5504, 5507, 5509, 5510, 5511, 5513, 5514, 5516, 5521, 5522, 5524, 5530, 5533, 5535, 5537, 5539, 5545, 5550, 5554, 5555, 5556, 5562, 5563, 5566, 5571, 5574, 5580, 5585, 5586, 5587, 5589, 5594, 5595, 5600, 5602, 5608, 5610, 5618, 5622, 5627, 5629, 5630, 5632, 5634, 5635, 5639, 5640, 5643, 5644, 5645, 5646, 5647, 5648, 5785, 5790, 5814, 5821, 5827, 5830, 5836, 5840, 5845, 5852, 5866, 5870, 5876, 5901, 5909, 5914, 5920, 5930, 5934, 5939, 5944, 5964, 5970, 5972, 5981, 5986, 5997, 5998, 6006, 6015, 6016, 6018, 6022, 6031, 6033, 6035, 6037, 6045, 6047, 6053, 6054, 6056, 6060, 6067, 6072, 6076, 6079, 6087, 6093, 6099, 6108, 6119, 6126, 6134, 6158, 6162, 6168, 6179, 6200, 6204, 6227, 6236, 6237, 6250, 6262, 6297, 6299, 6307, 6317, 6320, 6327, 6328, 6329, 6331, 6348, 6353, 6354, 6365, 6367, 6368, 6373, 6374, 6390, 6401, 6403, 6409, 6414, 6423, 6424, 6430, 6436, 6437, 6440, 6449, 6463, 6465, 6469, 6503, 6507, 6513, 6521, 6532, 6537, 6545, 6550, 6555, 6562, 6572, 6587, 6593, 6629, 6635, 6664, 6669, 6674, 6681, 6693, 6703, 6713, 6718, 6723, 6726, 6736, 6771, 6777, 6789, 6792, 6803, 6830, 6835, 6858, 6862, 6871, 6885, 6889, 6893, 6896, 6909, 6914, 6921, 6929, 6942, 6953, 6959, 6984, 6987, 7010, 7030, 7033, 7064, 7067, 7072, 7078, 7080, 7088, 7096, 7099, 7107, 7109, 7112, 7126, 7160, 7193, 7197, 7204, 7208, 7210, 7211, 7225, 7226, 7229, 7237, 7250, 7251, 8285, 8287, 8290, 8293, 8301, 8308, 8314, 8316, 8323, 8329, 8339, 8349, 8350, 8352, 8353, 8354, 8355, 8360, 8362, 8363, 8364, 8366, 8367, 8368, 8369, 8370, 8371, 8372, 8373, 8376, 8379, 8380, 8381, 8382, 8383, 8386, 8391, 8392, 8394, 8398, 8404, 8411, 8412, 8413, 8418, 8420, 8421, 8422, 8423, 8424, 8425, 8430, 8431, 8432, 8435, 8436, 8438, 8442, 8452, 8457, 8463, 8464, 8475, 8478, 8496, 8497, 8501, 8506, 8510, 8513, 8518, 8520, 8525, 8539, 8541, 8544, 8547, 8550, 8553, 8556, 8560, 8561, 8563, 8566, 8568, 8572, 8575, 8577, 8580, 8583, 8584, 8585, 8588, 8590, 8592, 8595, 8597, 8599, 8602, 8606, 8607, 8609, 8611, 8613, 8614, 8616, 8618, 8619, 8622, 8625, 8627, 8630, 8632, 8637, 8639, 8641, 8643, 8644, 8658, 8678, 8680, 8685, 8693, 8696, 8700, 8701, 8706, 8708, 8709, 8711, 8713, 8717, 8722, 8724, 8725, 8728, 8729, 8730, 8733, 8736, 8737, 8741, 8742, 8745, 8747, 8750, 9084, 9085, 9086, 9087, 9088, 9090, 9091, 9102, 9103, 9104, 9105, 9107, 9110, 9112, 9113, 9114, 9115, 9116, 9117, 9118, 9119, 9121, 9123, 9125, 9126, 9128, 9129, 9130, 9131, 9134, 9135, 9139, 9140, 9142, 9148, 9150, 9151, 9152, 9153, 9154, 9155, 9156]

In [ ]:
df = df.loc[response14_idx]
df = df[df['response_14'].notna()]
df['response_14'].isna().sum()

In [ ]:
df.shape

In [231]:
df.to_csv(#save response 14 specialist data)



# XGBoost 

In [ ]:
df = pd.read_csv(#read response 14 specialist data)

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
           'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'waist_to_hip_ratio',
            'idf_metabolic_syndrome',
            'eGFR',
            'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_IGA',
            'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            'FIB4',
            'NFS',
            'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            'BARD',
            'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_14'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_14 = NaN
df = df[features]
df = df[df['response_14'].notna()]

df['response_14'].value_counts()

In [ ]:
df.shape
df.columns

In [ ]:
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'Clinically Significant Fibrosis']
df



In [81]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Create train and test sets  
X = df.iloc[:,:35]
y = df['Clinically Significant Fibrosis']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

In [240]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB - Specialist Variables - Clinically Significant Fibrosis")
plt.show()



## XGBoost + MICE 

In [ ]:
df = pd.read_csv(#read response 14 imputed specialist data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'Clinically Significant Fibrosis']
df.shape

In [88]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Create train and test sets  
X = df.iloc[:,:35]
y = df['Clinically Significant Fibrosis']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

In [251]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB + MICE - Specialist Variables - Clinically Significant Fibrosis")
plt.show()



## XGBoost + MICE + SMOTE

In [93]:
import imblearn
from imblearn.over_sampling import SMOTE

In [ ]:
df = pd.read_csv(#read response 14 imputed specialist data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'Clinically Significant Fibrosis']
df.shape

In [ ]:
#Create train and test sets
X = df.iloc[:,:35]
y = df['Clinically Significant Fibrosis']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

In [96]:
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

In [ ]:
unique, counts = np.unique(y_train_res, return_counts=True)

print(np.asarray((unique, counts)).T)

In [ ]:
print(y_test.value_counts())

In [99]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train_res, y_train_res)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train_res,y_train_res)

In [265]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train_res, 
    y = y_train_res, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train_res
)

shap_values = explainer.shap_values(X_train_res)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train_res, plot_type="bar",show=False)
plt.title("XGB + MICE + SMOTE - Specialist Variables - Clinically Significant Fibrosis")
plt.show()



# Response 15 - Advanced Fibrosis (Histology Confirmed)

In [269]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

In [ ]:
features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
           'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'waist_to_hip_ratio',
            'idf_metabolic_syndrome',
            'eGFR',
            'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_IGA',
            'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            'FIB4',
            'NFS',
            'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            'BARD',
            'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_15'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_15 = NaN
df = df[features]
df = df[df['response_15'].notna()]

df['response_15'].value_counts()

In [271]:
#response 15 specialist index

response15_idx = [156, 218, 224, 230, 231, 235, 251, 256, 258, 494, 500, 508, 513, 518, 524, 529, 533, 541, 545, 551, 555, 566, 567, 705, 714, 716, 721, 722, 730, 733, 738, 745, 759, 762, 803, 814, 849, 867, 871, 905, 906, 910, 912, 948, 962, 965, 996, 1032, 1034, 1038, 1039, 1042, 1044, 1047, 1051, 1054, 1055, 1056, 1059, 1087, 1106, 1143, 1144, 1162, 1185, 1219, 1273, 1305, 1308, 1309, 1322, 1327, 1371, 1372, 1374, 1379, 1380, 1385, 1390, 1391, 1395, 1396, 1399, 2088, 2092, 2095, 2098, 2101, 2105, 4126, 4202, 4205, 4441, 4445, 4521, 4522, 4523, 4526, 4529, 4531, 4532, 4534, 4535, 4537, 4538, 4539, 4540, 4542, 4543, 4544, 4546, 4547, 4550, 4551, 4552, 4554, 4555, 4556, 4558, 4559, 4560, 4561, 4565, 4568, 4569, 4571, 4574, 4575, 4577, 4579, 4581, 4589, 4593, 4596, 4601, 4608, 4615, 4617, 4618, 4619, 4621, 4623, 4626, 4627, 4629, 4638, 4639, 4641, 4643, 4646, 4647, 4653, 4656, 4657, 4659, 4660, 4661, 4662, 4664, 4665, 4683, 4684, 4688, 4692, 4693, 4695, 4696, 5169, 5174, 5176, 5178, 5184, 5195, 5196, 5211, 5214, 5215, 5216, 5219, 5224, 5227, 5229, 5230, 5235, 5243, 5256, 5259, 5273, 5275, 5281, 5287, 5294, 5297, 5308, 5330, 5341, 5345, 5347, 5354, 5363, 5364, 5374, 5375, 5377, 5391, 5393, 5414, 5419, 5449, 5484, 5485, 5502, 5504, 5507, 5509, 5510, 5511, 5513, 5514, 5516, 5521, 5522, 5524, 5530, 5533, 5535, 5537, 5539, 5545, 5550, 5554, 5555, 5556, 5562, 5563, 5566, 5571, 5574, 5580, 5585, 5586, 5587, 5589, 5594, 5595, 5600, 5602, 5608, 5610, 5618, 5622, 5627, 5629, 5630, 5632, 5634, 5635, 5639, 5640, 5643, 5644, 5645, 5646, 5647, 5648, 5785, 5790, 5814, 5821, 5827, 5830, 5836, 5840, 5845, 5852, 5866, 5870, 5876, 5901, 5909, 5914, 5920, 5930, 5934, 5939, 5944, 5964, 5970, 5972, 5981, 5986, 5997, 5998, 6006, 6015, 6016, 6018, 6022, 6031, 6033, 6035, 6037, 6045, 6047, 6053, 6054, 6056, 6060, 6067, 6072, 6076, 6079, 6087, 6093, 6099, 6108, 6119, 6126, 6134, 6158, 6162, 6168, 6179, 6200, 6204, 6227, 6236, 6237, 6250, 6262, 6297, 6299, 6307, 6317, 6320, 6327, 6328, 6329, 6331, 6348, 6353, 6354, 6365, 6367, 6368, 6373, 6374, 6390, 6401, 6403, 6409, 6414, 6423, 6424, 6430, 6436, 6437, 6440, 6449, 6463, 6465, 6469, 6503, 6507, 6513, 6521, 6532, 6537, 6545, 6550, 6555, 6562, 6572, 6587, 6593, 6629, 6635, 6664, 6669, 6674, 6681, 6693, 6703, 6713, 6718, 6723, 6726, 6736, 6771, 6777, 6789, 6792, 6803, 6830, 6835, 6858, 6862, 6871, 6885, 6889, 6893, 6896, 6909, 6914, 6921, 6929, 6942, 6953, 6959, 6984, 6987, 7010, 7030, 7033, 7064, 7067, 7072, 7078, 7080, 7088, 7096, 7099, 7107, 7109, 7112, 7126, 7160, 7193, 7197, 7204, 7208, 7210, 7211, 7225, 7226, 7229, 7237, 7250, 7251, 8285, 8287, 8290, 8293, 8301, 8308, 8314, 8316, 8323, 8329, 8339, 8349, 8350, 8352, 8353, 8354, 8355, 8360, 8362, 8363, 8364, 8366, 8367, 8368, 8369, 8370, 8371, 8372, 8373, 8376, 8379, 8380, 8381, 8382, 8383, 8386, 8391, 8392, 8394, 8398, 8404, 8411, 8412, 8413, 8418, 8420, 8421, 8422, 8423, 8424, 8425, 8430, 8431, 8432, 8435, 8436, 8438, 8442, 8452, 8457, 8463, 8464, 8475, 8478, 8496, 8497, 8501, 8506, 8510, 8513, 8518, 8520, 8525, 8539, 8541, 8544, 8547, 8550, 8553, 8556, 8560, 8561, 8563, 8566, 8568, 8572, 8575, 8577, 8580, 8583, 8584, 8585, 8588, 8590, 8592, 8595, 8597, 8599, 8602, 8606, 8607, 8609, 8611, 8613, 8614, 8616, 8618, 8619, 8622, 8625, 8627, 8630, 8632, 8637, 8639, 8641, 8643, 8644, 8658, 8678, 8680, 8685, 8693, 8696, 8700, 8701, 8706, 8708, 8709, 8711, 8713, 8717, 8722, 8724, 8725, 8728, 8729, 8730, 8733, 8736, 8737, 8741, 8742, 8745, 8747, 8750, 9084, 9085, 9086, 9087, 9088, 9090, 9091, 9102, 9103, 9104, 9105, 9107, 9110, 9112, 9113, 9114, 9115, 9116, 9117, 9118, 9119, 9121, 9123, 9125, 9126, 9128, 9129, 9130, 9131, 9134, 9135, 9139, 9140, 9142, 9148, 9150, 9151, 9152, 9153, 9154, 9155, 9156]

In [ ]:
df = df.loc[response15_idx]
df = df[df['response_15'].notna()]
df['response_15'].isna().sum()

In [ ]:
df.shape

In [275]:
df.to_csv(#save response 15 specialist data)



## XGBoost

In [ ]:
df = pd.read_csv(#read response 15 specialist data)

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
           'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'waist_to_hip_ratio',
            'idf_metabolic_syndrome',
            'eGFR',
            'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_IGA',
            'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            'FIB4',
            'NFS',
            'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            'BARD',
            'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_15'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_15 = NaN
df = df[features]
df = df[df['response_15'].notna()]

df['response_15'].value_counts()

In [ ]:
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'Advanced Fibrosis (Histology Confirmed)']
df



In [106]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Create train and test sets  
X = df.iloc[:,:35]
y = df['Advanced Fibrosis (Histology Confirmed)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

In [284]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB - Specialist Variables - Advanced Fibrosis (Histology Confirmed)")
plt.show()



## XGBoost + MICE

In [ ]:
df = pd.read_csv(#read imputed specialist response 15 data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'Advanced Fibrosis (Histology Confirmed)']
df.shape

In [113]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Create train and test sets  
X = df.iloc[:,:35]
y = df['Advanced Fibrosis (Histology Confirmed)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

In [295]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB + MICE - Specialist Variables - Advanced Fibrosis (Histology Confirmed)")
plt.show()



## XGBoost + MICE + SMOTE

In [118]:
import imblearn
from imblearn.over_sampling import SMOTE

In [ ]:
df = pd.read_csv(#read imputed specialist response 15 data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'Advanced Fibrosis (Histology Confirmed)']
df.shape


In [120]:
#Create train and test sets
X = df.iloc[:,:35]
y = df['Advanced Fibrosis (Histology Confirmed)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

In [121]:
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

In [ ]:
unique, counts = np.unique(y_train_res, return_counts=True)

print(np.asarray((unique, counts)).T)

In [ ]:
print(y_test.value_counts())

In [124]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train_res, y_train_res)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train_res,y_train_res)

In [309]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train_res, 
    y = y_train_res, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train_res
)

shap_values = explainer.shap_values(X_train_res)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train_res, plot_type="bar",show=False)
plt.title("XGB + MICE + SMOTE - Specialist Variables - Advanced Fibrosis (Histology Confirmed)")
plt.show()



# Response 16 - Cirrhosis (Histology Confirmed)

In [313]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

In [ ]:
features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
           'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'waist_to_hip_ratio',
            'idf_metabolic_syndrome',
            'eGFR',
            'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_IGA',
            'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            'FIB4',
            'NFS',
            'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            'BARD',
            'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_16'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_16 = NaN
df = df[features]
df = df[df['response_16'].notna()]

df['response_16'].value_counts()

In [315]:
#response 16 specialist indexes
response16_idx = [156, 218, 224, 230, 231, 235, 251, 256, 258, 494, 500, 508, 513, 518, 524, 529, 533, 541, 545, 551, 555, 566, 567, 705, 714, 716, 721, 722, 730, 733, 738, 745, 759, 762, 803, 814, 849, 867, 871, 905, 906, 910, 912, 948, 962, 965, 996, 1032, 1034, 1038, 1039, 1042, 1044, 1047, 1051, 1054, 1055, 1056, 1059, 1087, 1106, 1143, 1144, 1162, 1185, 1219, 1273, 1305, 1308, 1309, 1322, 1327, 1371, 1372, 1374, 1379, 1380, 1385, 1390, 1391, 1395, 1396, 1399, 2088, 2092, 2095, 2098, 2101, 2105, 4126, 4202, 4205, 4441, 4445, 4521, 4522, 4523, 4526, 4529, 4531, 4532, 4534, 4535, 4537, 4538, 4539, 4540, 4542, 4543, 4544, 4546, 4547, 4550, 4551, 4552, 4554, 4555, 4556, 4558, 4559, 4560, 4561, 4565, 4568, 4569, 4571, 4574, 4575, 4577, 4579, 4581, 4589, 4593, 4596, 4601, 4608, 4615, 4617, 4618, 4619, 4621, 4623, 4626, 4627, 4629, 4638, 4639, 4641, 4643, 4646, 4647, 4653, 4656, 4657, 4659, 4660, 4661, 4662, 4664, 4665, 4683, 4684, 4688, 4692, 4693, 4695, 4696, 5169, 5174, 5176, 5178, 5184, 5195, 5196, 5211, 5214, 5215, 5216, 5219, 5224, 5227, 5229, 5230, 5235, 5243, 5256, 5259, 5273, 5275, 5281, 5287, 5294, 5297, 5308, 5330, 5341, 5345, 5347, 5354, 5363, 5364, 5374, 5375, 5377, 5391, 5393, 5414, 5419, 5449, 5484, 5485, 5502, 5504, 5507, 5509, 5510, 5511, 5513, 5514, 5516, 5521, 5522, 5524, 5530, 5533, 5535, 5537, 5539, 5545, 5550, 5554, 5555, 5556, 5562, 5563, 5566, 5571, 5574, 5580, 5585, 5586, 5587, 5589, 5594, 5595, 5600, 5602, 5608, 5610, 5618, 5622, 5627, 5629, 5630, 5632, 5634, 5635, 5639, 5640, 5643, 5644, 5645, 5646, 5647, 5648, 5785, 5790, 5814, 5821, 5827, 5830, 5836, 5840, 5845, 5852, 5866, 5870, 5876, 5901, 5909, 5914, 5920, 5930, 5934, 5939, 5944, 5964, 5970, 5972, 5981, 5986, 5997, 5998, 6006, 6015, 6016, 6018, 6022, 6031, 6033, 6035, 6037, 6045, 6047, 6053, 6054, 6056, 6060, 6067, 6072, 6076, 6079, 6087, 6093, 6099, 6108, 6119, 6126, 6134, 6158, 6162, 6168, 6179, 6200, 6204, 6227, 6236, 6237, 6250, 6262, 6297, 6299, 6307, 6317, 6320, 6327, 6328, 6329, 6331, 6348, 6353, 6354, 6365, 6367, 6368, 6373, 6374, 6390, 6401, 6403, 6409, 6414, 6423, 6424, 6430, 6436, 6437, 6440, 6449, 6463, 6465, 6469, 6503, 6507, 6513, 6521, 6532, 6537, 6545, 6550, 6555, 6562, 6572, 6587, 6593, 6629, 6635, 6664, 6669, 6674, 6681, 6693, 6703, 6713, 6718, 6723, 6726, 6736, 6771, 6777, 6789, 6792, 6803, 6830, 6835, 6858, 6862, 6871, 6885, 6889, 6893, 6896, 6909, 6914, 6921, 6929, 6942, 6953, 6959, 6984, 6987, 7010, 7030, 7033, 7064, 7067, 7072, 7078, 7080, 7088, 7096, 7099, 7107, 7109, 7112, 7126, 7160, 7193, 7197, 7204, 7208, 7210, 7211, 7225, 7226, 7229, 7237, 7250, 7251, 8285, 8287, 8290, 8293, 8301, 8308, 8314, 8316, 8323, 8329, 8339, 8349, 8350, 8352, 8353, 8354, 8355, 8360, 8362, 8363, 8364, 8366, 8367, 8368, 8369, 8370, 8371, 8372, 8373, 8376, 8379, 8380, 8381, 8382, 8383, 8386, 8391, 8392, 8394, 8398, 8404, 8411, 8412, 8413, 8418, 8420, 8421, 8422, 8423, 8424, 8425, 8430, 8431, 8432, 8435, 8436, 8438, 8442, 8452, 8457, 8463, 8464, 8475, 8478, 8496, 8497, 8501, 8506, 8510, 8513, 8518, 8520, 8525, 8539, 8541, 8544, 8547, 8550, 8553, 8556, 8560, 8561, 8563, 8566, 8568, 8572, 8575, 8577, 8580, 8583, 8584, 8585, 8588, 8590, 8592, 8595, 8597, 8599, 8602, 8606, 8607, 8609, 8611, 8613, 8614, 8616, 8618, 8619, 8622, 8625, 8627, 8630, 8632, 8637, 8639, 8641, 8643, 8644, 8658, 8678, 8680, 8685, 8693, 8696, 8700, 8701, 8706, 8708, 8709, 8711, 8713, 8717, 8722, 8724, 8725, 8728, 8729, 8730, 8733, 8736, 8737, 8741, 8742, 8745, 8747, 8750, 9084, 9085, 9086, 9087, 9088, 9090, 9091, 9102, 9103, 9104, 9105, 9107, 9110, 9112, 9113, 9114, 9115, 9116, 9117, 9118, 9119, 9121, 9123, 9125, 9126, 9128, 9129, 9130, 9131, 9134, 9135, 9139, 9140, 9142, 9148, 9150, 9151, 9152, 9153, 9154, 9155, 9156]

In [ ]:
df = df.loc[response16_idx]
df = df[df['response_16'].notna()]
df['response_16'].isna().sum()

In [ ]:
df.shape

In [ ]:
#Bar chart displaying level of completeness for each feature, n = 8550
null_values_bar = msno.bar(df,labels=True)
null_values_bar

In [319]:
df.to_csv(#save response 16 specialist data)



## XGBoost

In [ ]:
df = pd.read_csv(#read response 16 specialist data)

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
           'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'waist_to_hip_ratio',
            'idf_metabolic_syndrome',
            'eGFR',
            'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_IGA',
            'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            'FIB4',
            'NFS',
            'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            'BARD',
            'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_16'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_16 = NaN
df = df[features]
df = df[df['response_16'].notna()]

df['response_16'].value_counts()

In [ ]:
df.shape
df.columns

In [ ]:
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'Cirrhosis (Histology Confirmed)']
df



In [131]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Create train and test sets  
X = df.iloc[:,:35]
y = df['Cirrhosis (Histology Confirmed)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

In [328]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB - Specialist Variables - Cirrhosis (Histology Confirmed)")
plt.show()



## XGBoost + MICE

In [ ]:
df = pd.read_csv(#read imputed specialist response 16 data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'Cirrhosis (Histology Confirmed)']
df.shape

In [138]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Create train and test sets  
X = df.iloc[:,:35]
y = df['Cirrhosis (Histology Confirmed)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

In [339]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB + MICE - Specialist Variables - Cirrhosis (Histology Confirmed)")
plt.show()



## XGBoost + MICE + SMOTE

In [143]:
import imblearn
from imblearn.over_sampling import SMOTE

In [ ]:
df = pd.read_csv(#read response 16 imputed specialist data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'Cirrhosis (Histology Confirmed)']
df.shape

In [145]:
#Create train and test sets
X = df.iloc[:,:35]
y = df['Cirrhosis (Histology Confirmed)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

In [146]:
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

In [ ]:
unique, counts = np.unique(y_train_res, return_counts=True)

print(np.asarray((unique, counts)).T)

In [ ]:
print(y_test.value_counts())

In [149]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train_res, y_train_res)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train_res,y_train_res)

In [353]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train_res, 
    y = y_train_res, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train_res
)

shap_values = explainer.shap_values(X_train_res)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train_res, plot_type="bar",show=False)
plt.title("XGB + MICE + SMOTE - Specialist Variables - Cirrhosis (Histology Confirmed)")
plt.show()



# Response 17 - Advanced Fibrosis (Histology and Clinically Confirmed)

In [357]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

In [ ]:
features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
           'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'waist_to_hip_ratio',
            'idf_metabolic_syndrome',
            'eGFR',
            'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_IGA',
            'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            'FIB4',
            'NFS',
            'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            'BARD',
            'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_17'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_17 = NaN
df = df[features]
df = df[df['response_17'].notna()]

df['response_17'].value_counts()

In [359]:
#response 17 specialist indexes
response17_idx = [156, 218, 224, 230, 231, 235, 251, 256, 258, 494, 500, 508, 513, 518, 524, 529, 533, 541, 545, 551, 555, 566, 567, 705, 714, 716, 721, 722, 730, 733, 738, 745, 759, 762, 803, 814, 849, 867, 871, 905, 906, 910, 912, 948, 962, 965, 996, 1032, 1034, 1038, 1039, 1042, 1044, 1047, 1051, 1054, 1055, 1056, 1059, 1087, 1106, 1143, 1144, 1162, 1185, 1219, 1273, 1305, 1308, 1309, 1322, 1327, 1371, 1372, 1374, 1379, 1380, 1385, 1390, 1391, 1395, 1396, 1399, 2088, 2092, 2095, 2098, 2101, 2105, 4126, 4202, 4205, 4441, 4445, 4521, 4522, 4523, 4526, 4529, 4531, 4532, 4534, 4535, 4537, 4538, 4539, 4540, 4542, 4543, 4544, 4546, 4547, 4550, 4551, 4552, 4554, 4555, 4556, 4558, 4559, 4560, 4561, 4565, 4568, 4569, 4571, 4574, 4575, 4577, 4579, 4581, 4589, 4593, 4596, 4601, 4608, 4615, 4617, 4618, 4619, 4621, 4623, 4626, 4627, 4629, 4638, 4639, 4641, 4643, 4646, 4647, 4653, 4656, 4657, 4659, 4660, 4661, 4662, 4664, 4665, 4683, 4684, 4688, 4692, 4693, 4695, 4696, 5169, 5174, 5176, 5178, 5184, 5195, 5196, 5211, 5214, 5215, 5216, 5219, 5224, 5227, 5229, 5230, 5235, 5243, 5256, 5259, 5263, 5268, 5269, 5273, 5275, 5281, 5287, 5294, 5297, 5308, 5330, 5341, 5345, 5347, 5354, 5363, 5364, 5374, 5375, 5377, 5391, 5393, 5414, 5419, 5449, 5484, 5485, 5502, 5504, 5507, 5509, 5510, 5511, 5513, 5514, 5516, 5521, 5522, 5524, 5530, 5533, 5535, 5537, 5539, 5541, 5545, 5550, 5554, 5555, 5556, 5562, 5563, 5566, 5571, 5574, 5580, 5585, 5586, 5587, 5589, 5594, 5595, 5600, 5602, 5608, 5610, 5618, 5622, 5627, 5629, 5630, 5632, 5634, 5635, 5639, 5640, 5643, 5644, 5645, 5646, 5647, 5648, 5785, 5790, 5814, 5821, 5827, 5830, 5836, 5840, 5845, 5852, 5866, 5870, 5876, 5901, 5909, 5914, 5920, 5930, 5934, 5939, 5944, 5964, 5970, 5972, 5981, 5986, 5997, 5998, 6006, 6015, 6016, 6018, 6022, 6031, 6033, 6035, 6037, 6045, 6047, 6053, 6054, 6056, 6060, 6067, 6072, 6076, 6079, 6087, 6093, 6099, 6108, 6119, 6126, 6134, 6158, 6162, 6168, 6179, 6200, 6204, 6227, 6236, 6237, 6250, 6262, 6297, 6299, 6307, 6315, 6317, 6320, 6327, 6328, 6329, 6331, 6348, 6353, 6354, 6365, 6367, 6368, 6373, 6374, 6390, 6401, 6403, 6409, 6414, 6423, 6424, 6430, 6436, 6437, 6440, 6449, 6463, 6465, 6469, 6503, 6507, 6513, 6521, 6532, 6537, 6545, 6550, 6555, 6562, 6572, 6587, 6593, 6629, 6635, 6664, 6669, 6674, 6681, 6693, 6703, 6713, 6718, 6723, 6726, 6736, 6771, 6777, 6789, 6792, 6803, 6830, 6835, 6858, 6862, 6871, 6885, 6889, 6893, 6896, 6909, 6914, 6921, 6929, 6942, 6953, 6959, 6984, 6987, 7001, 7010, 7030, 7033, 7064, 7067, 7072, 7078, 7080, 7088, 7096, 7099, 7107, 7109, 7112, 7126, 7160, 7193, 7197, 7204, 7208, 7210, 7211, 7225, 7226, 7229, 7237, 7250, 7251, 8285, 8287, 8290, 8293, 8301, 8308, 8314, 8316, 8323, 8329, 8339, 8349, 8350, 8352, 8353, 8354, 8355, 8360, 8362, 8363, 8364, 8366, 8367, 8368, 8369, 8370, 8371, 8372, 8373, 8376, 8379, 8380, 8381, 8382, 8383, 8386, 8391, 8392, 8394, 8398, 8404, 8411, 8412, 8413, 8418, 8420, 8421, 8422, 8423, 8424, 8425, 8430, 8431, 8432, 8435, 8436, 8438, 8442, 8452, 8457, 8463, 8464, 8475, 8478, 8496, 8497, 8501, 8506, 8510, 8513, 8518, 8520, 8525, 8539, 8541, 8544, 8547, 8550, 8553, 8556, 8560, 8561, 8563, 8566, 8568, 8572, 8575, 8577, 8580, 8583, 8584, 8585, 8588, 8590, 8592, 8595, 8597, 8599, 8602, 8606, 8607, 8609, 8611, 8613, 8614, 8616, 8618, 8619, 8622, 8625, 8627, 8630, 8632, 8637, 8639, 8641, 8643, 8644, 8658, 8678, 8680, 8685, 8693, 8696, 8699, 8700, 8701, 8706, 8708, 8709, 8711, 8713, 8715, 8717, 8722, 8724, 8725, 8728, 8729, 8730, 8733, 8736, 8737, 8741, 8742, 8745, 8747, 8750, 9084, 9085, 9086, 9087, 9088, 9090, 9091, 9102, 9103, 9104, 9105, 9107, 9110, 9112, 9113, 9114, 9115, 9116, 9117, 9118, 9119, 9121, 9123, 9125, 9126, 9128, 9129, 9130, 9131, 9134, 9135, 9139, 9140, 9142, 9148, 9150, 9151, 9152, 9153, 9154, 9155, 9156]

In [ ]:
df = df.loc[response17_idx]
df = df[df['response_17'].notna()]
df['response_17'].isna().sum()

In [ ]:
df.shape

In [363]:
df.to_csv(#save response 17 specialist data)



# XGBoost

In [ ]:
df = pd.read_csv(#read response 17 specialist data)

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
           'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'waist_to_hip_ratio',
            'idf_metabolic_syndrome',
            'eGFR',
            'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_IGA',
            'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            'FIB4',
            'NFS',
            'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            'BARD',
            'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_17'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_17 = NaN
df = df[features]
df = df[df['response_17'].notna()]

df['response_17'].value_counts()

In [ ]:
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'Advanced Fibrosis (Histology & Clinical)']
df


In [156]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Create train and test sets  
X = df.iloc[:,:35]
y = df['Advanced Fibrosis (Histology & Clinical)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

In [372]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB - Specialist Variables - Advanced Fibrosis (Histology & Clinically Confirmed)")
plt.show()



## XGBoost + MICE

In [ ]:
df = pd.read_csv(#read response 17 imputed specialist data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'Advanced Fibrosis (Histology & Clinical)']
df.shape


In [ ]:
#Bar chart displaying level of completeness for each feature, n = 8550
null_values_bar = msno.bar(df,labels=True)
null_values_bar

In [163]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Create train and test sets  
X = df.iloc[:,:35]
y = df['Advanced Fibrosis (Histology & Clinical)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

In [383]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB + MICE - Specialist Variables - Advanced Fibrosis (Histology & Clinically Confirmed)")
plt.show()



## XGBoost + MICE + SMOTE

In [168]:
import imblearn
from imblearn.over_sampling import SMOTE

In [ ]:
df = pd.read_csv(#read imputed specialist response 17 data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'Advanced Fibrosis (Histology & Clinical)']
df.shape

In [170]:
#Create train and test sets
X = df.iloc[:,:35]
y = df['Advanced Fibrosis (Histology & Clinical)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

In [171]:
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

In [ ]:
unique, counts = np.unique(y_train_res, return_counts=True)

print(np.asarray((unique, counts)).T)

In [ ]:
print(y_test.value_counts())

In [174]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train_res, y_train_res)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train_res,y_train_res)

In [397]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train_res, 
    y = y_train_res, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train_res
)

shap_values = explainer.shap_values(X_train_res)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train_res, plot_type="bar",show=False)
plt.title("XGB + MICE + SMOTE - Specialist Variables - Advanced Fibrosis (Histology & Clinically Confirmed)")
plt.show()



# Response 18 - Cirrhosis (Histology & Clinically Confirmed)

In [401]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

In [ ]:
features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
           'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'waist_to_hip_ratio',
            'idf_metabolic_syndrome',
            'eGFR',
            'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_IGA',
            'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            'FIB4',
            'NFS',
            'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            'BARD',
            'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_18'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_18 = NaN
df = df[features]
df = df[df['response_18'].notna()]

df['response_18'].value_counts()

In [403]:
#response 18 specialist indexes
response18_idx = [156, 218, 224, 230, 231, 235, 251, 256, 258, 494, 500, 508, 513, 518, 524, 529, 533, 541, 545, 551, 555, 566, 567, 705, 714, 716, 721, 722, 730, 733, 738, 745, 759, 762, 803, 814, 849, 867, 871, 905, 906, 910, 912, 948, 962, 965, 996, 1032, 1034, 1038, 1039, 1042, 1044, 1047, 1051, 1054, 1055, 1056, 1059, 1087, 1106, 1143, 1144, 1162, 1185, 1219, 1273, 1305, 1308, 1309, 1322, 1327, 1371, 1372, 1374, 1379, 1380, 1385, 1390, 1391, 1395, 1396, 1399, 2088, 2092, 2095, 2098, 2101, 2105, 4126, 4202, 4205, 4441, 4445, 4521, 4522, 4523, 4526, 4529, 4531, 4532, 4534, 4535, 4537, 4538, 4539, 4540, 4542, 4543, 4544, 4546, 4547, 4550, 4551, 4552, 4554, 4555, 4556, 4558, 4559, 4560, 4561, 4565, 4568, 4569, 4571, 4574, 4575, 4577, 4579, 4581, 4589, 4593, 4596, 4601, 4608, 4615, 4617, 4618, 4619, 4621, 4623, 4626, 4627, 4629, 4638, 4639, 4641, 4643, 4646, 4647, 4653, 4656, 4657, 4659, 4660, 4661, 4662, 4664, 4665, 4683, 4684, 4688, 4692, 4693, 4695, 4696, 5169, 5174, 5176, 5178, 5184, 5195, 5196, 5211, 5214, 5215, 5216, 5219, 5224, 5227, 5229, 5230, 5235, 5243, 5256, 5259, 5263, 5268, 5269, 5273, 5275, 5281, 5287, 5294, 5297, 5308, 5330, 5341, 5345, 5347, 5354, 5363, 5364, 5374, 5375, 5377, 5391, 5393, 5414, 5419, 5449, 5484, 5485, 5502, 5504, 5507, 5509, 5510, 5511, 5513, 5514, 5516, 5521, 5522, 5524, 5530, 5533, 5535, 5537, 5539, 5541, 5545, 5550, 5554, 5555, 5556, 5562, 5563, 5566, 5571, 5574, 5580, 5585, 5586, 5587, 5589, 5594, 5595, 5600, 5602, 5608, 5610, 5618, 5622, 5627, 5629, 5630, 5632, 5634, 5635, 5639, 5640, 5643, 5644, 5645, 5646, 5647, 5648, 5785, 5790, 5814, 5821, 5827, 5830, 5836, 5840, 5845, 5852, 5866, 5870, 5876, 5901, 5909, 5914, 5920, 5930, 5934, 5939, 5944, 5964, 5970, 5972, 5981, 5986, 5997, 5998, 6006, 6015, 6016, 6018, 6022, 6031, 6033, 6035, 6037, 6045, 6047, 6053, 6054, 6056, 6060, 6067, 6072, 6076, 6079, 6087, 6093, 6099, 6108, 6119, 6126, 6134, 6158, 6162, 6168, 6179, 6200, 6204, 6227, 6236, 6237, 6250, 6262, 6297, 6299, 6307, 6315, 6317, 6320, 6327, 6328, 6329, 6331, 6348, 6353, 6354, 6365, 6367, 6368, 6373, 6374, 6390, 6401, 6403, 6409, 6414, 6423, 6424, 6430, 6436, 6437, 6440, 6449, 6463, 6465, 6469, 6503, 6507, 6513, 6521, 6532, 6537, 6545, 6550, 6555, 6562, 6572, 6587, 6593, 6629, 6635, 6664, 6669, 6674, 6681, 6693, 6703, 6713, 6718, 6723, 6726, 6736, 6771, 6777, 6789, 6792, 6803, 6830, 6835, 6858, 6862, 6871, 6885, 6889, 6893, 6896, 6909, 6914, 6921, 6929, 6942, 6953, 6959, 6984, 6987, 7001, 7010, 7030, 7033, 7064, 7067, 7072, 7078, 7080, 7088, 7096, 7099, 7107, 7109, 7112, 7126, 7160, 7193, 7197, 7204, 7208, 7210, 7211, 7225, 7226, 7229, 7237, 7250, 7251, 8285, 8287, 8290, 8293, 8301, 8308, 8314, 8316, 8323, 8329, 8339, 8349, 8350, 8352, 8353, 8354, 8355, 8360, 8362, 8363, 8364, 8366, 8367, 8368, 8369, 8370, 8371, 8372, 8373, 8376, 8379, 8380, 8381, 8382, 8383, 8386, 8391, 8392, 8394, 8398, 8404, 8411, 8412, 8413, 8418, 8420, 8421, 8422, 8423, 8424, 8425, 8430, 8431, 8432, 8435, 8436, 8438, 8442, 8452, 8457, 8463, 8464, 8475, 8478, 8496, 8497, 8501, 8506, 8510, 8513, 8518, 8520, 8525, 8539, 8541, 8544, 8547, 8550, 8553, 8556, 8560, 8561, 8563, 8566, 8568, 8572, 8575, 8577, 8580, 8583, 8584, 8585, 8588, 8590, 8592, 8595, 8597, 8599, 8602, 8606, 8607, 8609, 8611, 8613, 8614, 8616, 8618, 8619, 8622, 8625, 8627, 8630, 8632, 8637, 8639, 8641, 8643, 8644, 8658, 8678, 8680, 8685, 8693, 8696, 8699, 8700, 8701, 8706, 8708, 8709, 8711, 8713, 8715, 8717, 8722, 8724, 8725, 8728, 8729, 8730, 8733, 8736, 8737, 8741, 8742, 8745, 8747, 8750, 9084, 9085, 9086, 9087, 9088, 9090, 9091, 9102, 9103, 9104, 9105, 9107, 9110, 9112, 9113, 9114, 9115, 9116, 9117, 9118, 9119, 9121, 9123, 9125, 9126, 9128, 9129, 9130, 9131, 9134, 9135, 9139, 9140, 9142, 9148, 9150, 9151, 9152, 9153, 9154, 9155, 9156]

In [ ]:
df = df.loc[response18_idx]
df = df[df['response_18'].notna()]
df['response_18'].isna().sum()

In [ ]:
df.shape

In [407]:
df.to_csv(#save response 18 specialist data)



## XGBoost

In [ ]:
df = pd.read_csv(#read response 18 specialist data)

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
           'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'waist_to_hip_ratio',
            'idf_metabolic_syndrome',
            'eGFR',
            'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_IGA',
            'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            'FIB4',
            'NFS',
            'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            'BARD',
            'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_18'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_18 = NaN
df = df[features]
df = df[df['response_18'].notna()]

df['response_18'].value_counts()

In [ ]:
df.shape
df.columns

In [ ]:
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'Cirrhosis (Histology & Clinically Confirmed)']
df



In [181]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Create train and test sets  
X = df.iloc[:,:35]
y = df['Cirrhosis (Histology & Clinically Confirmed)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

In [416]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB - Specialist Variables - Cirrhosis (Histology & Clinically Confirmed)")
plt.show()



## XGBoost + MICE

In [ ]:
df = pd.read_csv(#read imputed specialist response 18 data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'Cirrhosis (Histology & Clinically Confirmed)']
df.shape


In [188]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Create train and test sets  
X = df.iloc[:,:35]
y = df['Cirrhosis (Histology & Clinically Confirmed)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

In [427]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB + MICE - Specialist Variables - Cirrhosis (Histology & Clinically Confirmed)")
plt.show()



## XGBoost + MICE + SMOTE

In [193]:
import imblearn
from imblearn.over_sampling import SMOTE

In [ ]:
df = pd.read_csv(#read imputed specialist response 18 data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'Cirrhosis (Histology & Clinically Confirmed)']
df.shape

In [195]:
#Create train and test sets
X = df.iloc[:,:35]
y = df['Cirrhosis (Histology & Clinically Confirmed)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

In [196]:
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

In [ ]:
unique, counts = np.unique(y_train_res, return_counts=True)

print(np.asarray((unique, counts)).T)

In [ ]:
print(y_test.value_counts())

In [199]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train_res, y_train_res)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train_res,y_train_res)

In [441]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train_res, 
    y = y_train_res, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train_res
)

shap_values = explainer.shap_values(X_train_res)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train_res, plot_type="bar",show=False)
plt.title("XGB + MICE + SMOTE - Specialist Variables - Cirrhosis (Histology & Clinically Confirmed)")
plt.show()



# Response 20 - At-Risk MASLD

In [445]:
#Only considering Baseline Event Types
df = pd.read_csv(#read data)
df = df[df['CPH_EV_EVENT_TYPE'] == 'Baseline']
df = df[df['TBL.METRICS..MET_ADM_COHORT_LITMUS'] == 'METACOHORT']

In [ ]:
features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
           'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'waist_to_hip_ratio',
            'idf_metabolic_syndrome',
            'eGFR',
            'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_IGA',
            'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            'FIB4',
            'NFS',
            'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            'BARD',
            'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_20'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_20 = NaN
df = df[features]
df = df[df['response_20'].notna()]

df['response_20'].value_counts()

In [ ]:
#response 20 specialist indexes
response20_idx = [156, 218, 224, 230, 231, 251, 256, 494, 500, 508, 513, 518, 524, 529, 533, 541, 545, 551, 555, 566, 567, 705, 714, 716, 721, 722, 730, 733, 738, 745, 759, 762, 803, 814, 849, 867, 871, 905, 906, 910, 912, 948, 962, 965, 996, 1032, 1034, 1038, 1039, 1042, 1044, 1047, 1051, 1055, 1056, 1059, 1087, 1106, 1143, 1144, 1185, 1273, 1305, 1308, 1309, 1322, 1327, 1371, 1372, 1374, 1379, 1380, 1390, 1391, 1395, 1396, 1399, 2088, 2092, 2095, 2101, 2105, 4126, 4202, 4205, 4445, 4521, 4522, 4523, 4526, 4529, 4531, 4532, 4534, 4535, 4537, 4538, 4539, 4540, 4542, 4543, 4546, 4547, 4550, 4551, 4552, 4554, 4555, 4558, 4559, 4560, 4561, 4565, 4568, 4569, 4575, 4577, 4581, 4589, 4593, 4596, 4601, 4608, 4615, 4617, 4618, 4619, 4621, 4623, 4627, 4629, 4638, 4639, 4641, 4643, 4646, 4647, 4653, 4657, 4659, 4660, 4661, 4662, 4664, 4665, 4683, 4684, 4688, 4692, 4693, 4695, 4696, 5169, 5174, 5176, 5178, 5184, 5195, 5196, 5211, 5214, 5215, 5216, 5219, 5224, 5230, 5235, 5256, 5259, 5273, 5275, 5287, 5294, 5297, 5308, 5330, 5341, 5345, 5347, 5354, 5363, 5364, 5374, 5375, 5377, 5391, 5393, 5414, 5419, 5449, 5484, 5485, 5502, 5507, 5509, 5510, 5511, 5513, 5514, 5516, 5521, 5522, 5524, 5533, 5535, 5545, 5550, 5554, 5563, 5566, 5571, 5580, 5585, 5586, 5587, 5589, 5595, 5600, 5602, 5608, 5610, 5622, 5627, 5629, 5630, 5632, 5634, 5635, 5639, 5640, 5644, 5645, 5646, 5647, 5648, 5785, 5790, 5814, 5827, 5830, 5836, 5840, 5845, 5866, 5870, 5876, 5901, 5909, 5914, 5920, 5934, 5944, 5964, 5970, 5972, 5981, 5997, 5998, 6006, 6015, 6018, 6031, 6035, 6037, 6047, 6053, 6054, 6060, 6067, 6072, 6079, 6087, 6093, 6099, 6108, 6119, 6134, 6162, 6168, 6179, 6200, 6236, 6237, 6250, 6262, 6297, 6299, 6320, 6327, 6328, 6353, 6354, 6368, 6373, 6401, 6403, 6409, 6414, 6423, 6424, 6430, 6436, 6440, 6449, 6463, 6503, 6507, 6513, 6521, 6532, 6545, 6550, 6562, 6572, 6587, 6593, 6629, 6664, 6669, 6674, 6681, 6703, 6713, 6718, 6723, 6736, 6771, 6777, 6789, 6792, 6803, 6835, 6858, 6871, 6885, 6893, 6896, 6909, 6929, 6953, 6959, 6984, 6987, 7010, 7030, 7033, 7064, 7067, 7072, 7078, 7080, 7088, 7096, 7099, 7107, 7109, 7112, 7126, 7160, 7204, 7208, 7211, 7226, 7229, 7237, 7250, 7251, 8285, 8287, 8290, 8293, 8301, 8308, 8314, 8316, 8323, 8329, 8339, 8349, 8350, 8352, 8353, 8354, 8355, 8360, 8362, 8363, 8364, 8366, 8367, 8368, 8369, 8370, 8371, 8373, 8376, 8379, 8380, 8381, 8382, 8383, 8386, 8391, 8392, 8394, 8398, 8404, 8411, 8412, 8413, 8418, 8420, 8421, 8422, 8423, 8424, 8425, 8430, 8431, 8432, 8435, 8436, 8442, 8452, 8457, 8463, 8464, 8475, 8478, 8497, 8501, 8506, 8510, 8513, 8518, 8520, 8525, 8539, 8541, 8544, 8547, 8550, 8553, 8556, 8560, 8561, 8563, 8566, 8568, 8572, 8575, 8577, 8580, 8584, 8585, 8588, 8592, 8595, 8599, 8602, 8606, 8607, 8609, 8611, 8613, 8614, 8618, 8619, 8622, 8627, 8630, 8632, 8637, 8639, 8641, 8643, 8644, 8658, 8678, 8680, 8685, 8693, 8700, 8708, 8713, 8724, 8725, 8728, 8729, 8730, 8737, 8741, 8742, 8747, 9084, 9085, 9086, 9088, 9090, 9091, 9102, 9103, 9104, 9107, 9110, 9112, 9113, 9114, 9115, 9116, 9117, 9118, 9119, 9121, 9123, 9125, 9126, 9128, 9129, 9130, 9131, 9134, 9135, 9139, 9140, 9142, 9148, 9150, 9151, 9152, 9153, 9156]

In [ ]:

df = df.loc[response20_idx]
df = df[df['response_20'].notna()]
df['response_20'].isna().sum()

In [ ]:
df.shape

In [451]:
df.to_csv(#save response 20 specialist data)



## XGBoost

In [ ]:
df = pd.read_csv(#read response 20 specialist data)

features_list = {
    'std_clinical_features' :['CPH_EV_AGE_CALC',
           'TBL.PATIENT.INFO..PI_BL_GENDER',
           'CPH_EV_CI_BMI_CALC',
           'TBL.ALL.EVENTS..AE_SF_ALCO_XS',
            'insulin_resistance',
            'hypertensive',
            'waist_to_hip_ratio',
            'idf_metabolic_syndrome',
            'eGFR',
            'dyslipidaemia',
            'fibroscan_stiffness_reliable',
            'TBL.ALL.EVENTS..AE_BR_ALT_iuL',
            'TBL.ALL.EVENTS..AE_BR_AST_iuL',
            'TBL.ALL.EVENTS..AE_BR_GGT_iuL',
            'TBL.ALL.EVENTS..AE_BR_FERR_ugL',
            'TBL.ALL.EVENTS..AE_BR_PLT_109L',
            'TBL.ALL.EVENTS..AE_BR_CREAT_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_STG_mmolL_CALC',
           'TBL.ALL.EVENTS..AE_BR_ALBU_gL_CALC',
            'TBL.ALL.EVENTS..AE_BR_BILI_umolL_CALC',
            'TBL.ALL.EVENTS..AE_BR_IGA',
            'TBL.ALL.EVENTS..AE_CD_OSA'
                             ],
    
    'single_markers':[
            'LIT_NB_CK18_M30',
            'LIT_NB_CK18_M65',
            'LIT_NB_PRO_C3',
            'LIT_NB_PRO_C6'],
 
   
    'multi_markers':[
            'LIT_NB_ELF',
            'FIB4',
            'NFS',
            'APRI',
            'ADAPT',
            'FIBC3',
            'ABC3D',
            'BARD',
            'AST_ALT_Ratio'
           ],
    
    'target':[
        'response_20'
    ]
}
features = []
features = features + features_list['std_clinical_features']
features = features + features_list['single_markers']
features = features + features_list['multi_markers']
features = features + features_list['target']

#Remove all rows where response_20 = NaN
df = df[features]
df = df[df['response_20'].notna()]

df['response_20'].value_counts()

In [ ]:
df.shape
df.columns

In [ ]:
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'response_20']
df



In [206]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Create train and test sets  
X = df.iloc[:,:35]
y = df['response_20']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

In [ ]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB - Specialist Variables - At-Risk MASLD")
plt.show()



## XGBoost + MICE

In [ ]:
df = pd.read_csv(#imputed response 20 specialist data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'response_20']
df.shape

In [213]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Create train and test sets  
X = df.iloc[:,:35]
y = df['response_20']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train,y_train)

In [471]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train, 
    y = y_train, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train
)

shap_values = explainer.shap_values(X_train)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.title("XGB + MICE - Specialist Variables - At-Risk MASLD")
plt.show()



## XGBoost + MICE + SMOTE

In [218]:
import imblearn
from imblearn.over_sampling import SMOTE

In [ ]:
df = pd.read_csv(#read response 20 imputed specialist data)
df.columns = ['Age', 'Gender', 'BMI', 'Excessive Alcohol Consumption', 'Insulin Resistance', 'Hypertensive',
              'Waist-to-Hip Ratio',
              'Metabolic Syndrome', 'eGFR','Dyslipidaemia', 'Fibroscan Stiffness', 'ALT', 'AST', 'GGT', 'Ferritin', 'Platelets', 'Creatinine',
              'Serum Triglycerides', 'Albumin', 'Bilirubin', 'IgA', 'Obstructive Sleep Apnoea',
              'CK18-M30','CK18-M65','Pro-C3','Pro-C6','ELF','FIB4',
              'NFS', 'APRI','ADAPT','FIBC3','ABC3D','BARD','AST-ALT Ratio',
              'response_20']
df.shape

In [ ]:
#Create train and test sets
X = df.iloc[:,:35]
y = df['response_20']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

In [221]:
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

In [ ]:
unique, counts = np.unique(y_train_res, return_counts=True)

print(np.asarray((unique, counts)).T)

In [ ]:
print(y_test.value_counts())

In [224]:
#Specify XGBoost Classifiers static parameters
XGBCL_STATIC_PARAMS = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1.0,
    'max_delta_step': 0.0,
    'min_child_weight': 1.0,
    'missing': None,
    'n_jobs': -1,
    'objective': 'binary:logistic',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'random_state': 42,
    'scale_pos_weight': 1.0,
    'tree_method': 'auto'
}

#Specify number of k-fold cross validation
KFOLD_STATIC_PARAMS = {
    'n_splits': 5,  # At least 2
    'shuffle': True, 
    'random_state': 42
}

#Specify grid search static parameters
SEARCH_GRID_STATIC_PARAMS = {
    'n_jobs': -1, 
    'scoring': 'accuracy',  # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    'verbose': 1
}

#Grid search XGBoost parameters with following values / ranges
SEARCH_GRID_PARAMS = { 
    'colsample_bytree':[.75, .8, 1],
    'learning_rate':[0.001, 0.01, 0.1],
    #'gamma': [0.0, 1.0],
    'max_depth':[1,2,5,8,12],
    'n_estimators': list(range(50, 300, 50)),
    #'num_parallel_tree': [1, 2, 4, 8],
    #'reg_alpha': [0.0, 0.5, 1.0],
    #'reg_lambda': [0.0, 0.5, 1.0],
    #'subsample':[.75,1],
}

#Allocate the static XGBoost parameters to our XGBoost classifier

xgbcl = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method']
)

#Specify k-fold cross-validation
kfold = StratifiedKFold(
    n_splits = KFOLD_STATIC_PARAMS['n_splits'], 
    shuffle = KFOLD_STATIC_PARAMS['shuffle'], 
    random_state = KFOLD_STATIC_PARAMS['random_state']
)

#Define grid search
grid_search = GridSearchCV(
    estimator = xgbcl,
    param_grid = SEARCH_GRID_PARAMS, 
    cv = kfold,
    n_jobs = SEARCH_GRID_STATIC_PARAMS['n_jobs'],
    scoring = SEARCH_GRID_STATIC_PARAMS['scoring'],
    verbose = SEARCH_GRID_STATIC_PARAMS['verbose']
)

In [ ]:
#Begin grid search for best parameters in XGBoost classifier for X_train data fitted onto y_train data
start_time = time.time()
grid_result = grid_search.fit(X_train_res, y_train_res)
grid_search_time = time.time() - start_time
print(f'Training performed in {grid_search_time/60} minutes')
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
#Print XGBoost model that gave best accuracy score from grid search
model = xgb.XGBClassifier(
    base_score = XGBCL_STATIC_PARAMS['base_score'],
    booster = XGBCL_STATIC_PARAMS['booster'],
    colsample_bylevel = XGBCL_STATIC_PARAMS['colsample_bylevel'],
    max_delta_step = XGBCL_STATIC_PARAMS['max_delta_step'],
    min_child_weight = XGBCL_STATIC_PARAMS['min_child_weight'],
    missing = XGBCL_STATIC_PARAMS['missing'],
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    objective = XGBCL_STATIC_PARAMS['objective'],
    random_state = XGBCL_STATIC_PARAMS['random_state'],
    scale_pos_weight = XGBCL_STATIC_PARAMS['scale_pos_weight'],
    tree_method = XGBCL_STATIC_PARAMS['tree_method'],
    
    colsample_bytree = grid_result.best_params_['colsample_bytree'], 
    learning_rate = grid_result.best_params_['learning_rate'], 
    #gamma = grid_result.best_params_['gamma'],
    max_depth = grid_result.best_params_['max_depth'], 
    n_estimators = grid_result.best_params_['n_estimators'],
    #num_parallel_tree = grid_result.best_params_['num_parallel_tree'],
    #reg_alpha = grid_result.best_params_['reg_alpha'],
    #reg_lambda = grid_result.best_params_['reg_lambda'],
    #subsample = grid_result.best_params_['subsample'],

)
print(model)

In [ ]:
#Fit the model onto X_train and y_train data
model.fit(X_train_res,y_train_res)

In [485]:
#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(
    estimator = model, 
    X = X_train_res, 
    y = y_train_res, 
    cv = kfold, 
    n_jobs = XGBCL_STATIC_PARAMS['n_jobs'],
    scoring = ['accuracy', 'roc_auc', 'precision', 'recall', 'f1']
)

In [ ]:
#Print metrics to evaluate model. 
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)

print('AUC: ',       scores['test_roc_auc'].mean())
print('Accuracy: ',  scores['test_accuracy'].mean())
print('sensitivity:', sens)
print('specificity:', spec)
print('F1: ',        scores['test_f1'].mean())
print('Test set score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
import sklearn.metrics as metrics

# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#Add SHAP values

explainer = shap.TreeExplainer(
    model, 
    model_output='probability', 
    feature_dependence='interventional', 
    data=X_train_res
)

shap_values = explainer.shap_values(X_train_res)
np.set_printoptions(threshold=sys.maxsize)
shap_values

#Bar chart represents the mean SHAP averages, i.e. the average impact on the model's output magnitude.
shap.summary_plot(shap_values, X_train_res, plot_type="bar",show=False)
plt.title("XGB + MICE + SMOTE - Specialist Variables - At-Risk MASLD")
plt.show()

